## Part2 Feature Engineering

### ライブラリのインポート

In [1]:
import os
import re
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

### csvファイル読み取り用の共通の関数

In [2]:
def read_stat_csv(path_input,file):
    ''' ヘッダーの行数を探した後に、csvファイルを読み込む。 '''
    path_join = os.path.join(path_input,file)
    with open(path_join,"r", encoding="shift-jis") as file_text:
        skiprows = file_text.read(2000).replace('"','').split('\n').index('VALUE') + 1

    df = pd.read_csv(path_join, encoding="shift-jis", skiprows=skiprows)
    return df

In [3]:
def pivot_df(df,field_title,field_year='時間軸(年次)'):
    ''' Pivot tableを使って同一属性値毎に集計 '''
    df.loc[:,field_year] = df.loc[:,field_year].apply(lambda x: re.sub(r'[^\x00-\x7F]+','',x))
    field_list = [field_title,'area_code','value']
    df = df.loc[df.loc[:,field_year] == max(df.loc[:,field_year].unique()),:][field_list]
    try:
        df.loc[:,'area_code'] = df.loc[:,'area_code'].apply(int)
    except:
        df.loc[:,'area_code'] = df.loc[:,'area_code'].apply(str).str.rjust(5,'0')
    df = df.pivot(index='area_code',columns=field_title,values='value')
    return df

### 国勢調査データの読み取りと前処理

In [4]:
def rename_df_columns(df_concat):
    ''' 統計データの項目と項目番号をタイトルから分けて、Dictionaryに格納 '''
    
    df_columns = df_concat.columns.values.tolist()    
    list_fields = [[item.split('_')[0],item.split('_')[1]] for item in df_columns[1:]]
    list_fields.insert(0,np.repeat(df_columns[0],2).tolist())    
    list_code = []
    list_descr = []
    for items in list_fields:
        list_code.append(items[0].replace('#',''))
        list_descr.append(items[1].replace('#',''))
    
    dict_fields = dict(zip(list_code,list_descr)) 
    return dict_fields

In [5]:
def data_census(df_field_use,list_stats,path_input):
    ''' ダウンロードしたデータをロードし、テーブルを結合する。'''
    
    list_code = df_field_use['filename'].values.tolist()
    dict_df = dict((int(y.split('.csv')[0]),[]) for y in list_code)
    df_concat = pd.DataFrame()
    for i, file in enumerate(list_code):
        id_file = int(file.split('.csv')[0])
        
        # 202xx台と203xxの番号のファイルを読み込む
        if (id_file > 20200) & (id_file < 3e6):
            field_title = list_stats[list_stats['id']==id_file]['TITLE'].values[0]
            df = read_stat_csv(path_input,file)
            dict_df[id_file] = pivot_df(df,field_title,field_year='調査年')
            if i == 0:
                df_concat = dict_df[id_file]
            else:
                df_concat = pd.concat([df_concat,dict_df[id_file]],axis=1)
    
    df_concat.reset_index(inplace=True)
    df_concat = df_concat.replace(to_replace='-', value=np.nan)
    df_use = df_concat.loc[df_concat.isnull().sum(axis=1) < 100, df_concat.isnull().sum(axis=0) < 100]
    df_use.loc[:,'area_code'] = df_use['area_code'].apply(str).str.rjust(5,'0')
    df_use.dropna(inplace=True)
    df_use.replace(to_replace=np.nan, value=0)
    for item in list(df_use.columns)[1:]:
        df_use.loc[:,item] = pd.to_numeric(df_use[item])
    
    dict_fields = rename_df_columns(df_use)
    df_use.columns = list(dict_fields.values())
    df_use.set_index('area_code',inplace=True)
    return df_use

In [7]:
path_input = "./download/stats_city"
df_field_use = pd.read_csv('./df_field_use.csv',names=['filename','description'])
list_stats = pd.read_csv('download/all_stats_city.csv')
df_census = data_census(df_field_use,list_stats,path_input)

### 他の政府統計データの読み込みと前処理

'FEH_00200521.csv': APIリクエストからは手に入らなかったため、e-Statsのサイトから直接ダウンロード。学歴データを含む。 <br>
'0003355547.csv': 家族構成 <br>
'0003355315.csv': 住宅 <br>
'0003355963.csv': 世帯人数 <br>
'0003355489.csv': 世帯年収 <br>

In [8]:
def df_preprocess(df,str_query,field_name,flag):
    '''
    クエリーでフィルターをかけた後、Pivot tableで集計。その後、不要な文字列の削除とデータ型の調整。
    '''
    df_filtered = df.query(str_query)
    if flag == 0:
        list_columns = list(df.columns)
        list_use = list_columns[13:]
        list_use.insert(0,list_columns[4])
        df_piv = df_filtered.loc[:,list_use]
        df_piv = df_piv.replace(to_replace='***', value=np.nan)
        df_piv = df_piv.replace(to_replace='-', value=0)
        df_piv.dropna(inplace=True)
        for item in list(df_piv)[1:]:
            df_piv.loc[:,item] = df_piv.loc[:,item].apply(lambda x: str(x).replace(',','')).apply(int)
        df_piv.rename(columns={list_use[0]: 'area_code'},inplace=True)
        
    elif flag == 1:
        df_piv = df_filtered.loc[1:,['area_code','value']]
        df_piv.loc[:,'value'] = df_piv.loc[:,'value'].apply(float)
        
    else:
        df_piv = pivot_df(df_filtered,field_name)
        df_piv = df_piv.replace(to_replace='-', value=0).reset_index()
        df_piv_columns = list(df_piv.columns)
        for column in df_piv_columns:
            df_piv.loc[:,column] = df_piv.loc[:,column].apply(int)        
    
    df_piv.loc[:,'area_code'] = df_piv.loc[:,'area_code'].apply(str).str.rjust(5,'0')
    df_piv.set_index('area_code',inplace=True)

    if flag == 1:        
        df_piv.rename(columns={'value':field_name},inplace=True)
    return df_piv

In [9]:
def stats_other(path_input,filenames,str_queries,fields):
    ''' 5つのcsvファイルからデータを読み取り、前処理後に結合 '''
    
    for i, filename in enumerate(filenames):
        if i == 0:
            df = pd.read_csv(os.path.join(path_input,filenames[i]),skiprows=10)
            df_stats = df_preprocess(df,str_queries[i],fields[i],flags[i])
        else:
            df = read_stat_csv(path_input,filenames[i])
            if (i == 1) | (i == 2):
                for j in [0,1]:
                    df_tmp = df_preprocess(df,str_queries[i][j],fields[i][j],flags[i][j])
                    df_stats = pd.concat([df_stats,df_tmp],axis=1,join='inner')
            else:
                df_tmp = df_preprocess(df,str_queries[i],fields[i],flags[i])
                df_stats = pd.concat([df_stats,df_tmp],axis=1,join='inner')
    return df_stats

In [11]:
filenames = ['FEH_00200521.csv','0003355547.csv','0003355315.csv','0003355963.csv','0003355489.csv']
str_queries = ["年齢2=='総数（年齢）'&男女別2010=='総数（男女別）'&全域人口集中地区2010=='人口集中地区'",
               ["住宅の所有の関係=='総数'&家族類型=='総数'",
               "表章項目=='普通世帯数'&住宅の所有の関係=='総数'&子の居住地=='総数'"],
               ["表章項目=='住宅数'&住宅の種類=='専用住宅'&住宅の建て方=='総数'",
               "表章項目=='１住宅当たり延べ面積'&住宅の種類=='専用住宅'&住宅の建て方=='総数'&住宅の所有の関係=='総数'"],
               "表章項目=='１世帯当たり人員'&世帯の年間収入階級=='総数'&住宅の所有の関係=='総数'& \
                世帯の種類=='総数'&家計を主に支える者の従業上の地位=='総数'",
               "住宅の所有の関係 == '総数' & 家族類型 == '総数'"]
flags = [0,[2,2],[2,1],1,2]
fields = [None,['子の居住地','家族類型'],['住宅の所有の関係','住宅平均面積'],
           '平均世帯人員','世帯の年間収入階級']
df_stats = stats_other(path_input,filenames,str_queries,fields)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### GISデータ（シェープファイル）の読み取りと前処理
国土数値情報ダウンロードサービスの国土交通省国土政策局国土情報課 <br>
http://nlftp.mlit.go.jp/ksj/

QGISで一度読み込み、簡単な計算を施した後に、エクスポート。Geopandasによる処理も可能。

In [12]:
def data_gis(path_shapefile):
    ''' シェープファイルの読み込み '''
    list_shapefile = ["chika_station.shp","Iryo.shp","Station1.shp"]
    for i,filename in enumerate(list_shapefile):
        path = os.path.join(path_shapefile,filename)
        df_gpd = gpd.read_file(path)
        if i == 0:
            list_field = list(df_gpd.columns)[6:-1]
        elif i == 1:
            list_field = list(df_gpd.columns)[11:-2]
        elif i ==2:
            list_field = list(df_gpd.columns)[14:-1]
    
        df_gpd.set_index('JCODE',inplace=True)
        df_gpd = df_gpd.loc[:,list_field]
        
        if i == 0:
            df_gis = df_gpd
        else:
            df_gis = pd.concat([df_gis,df_gpd], axis=1, join='inner')
    
    df_gis.MEAN_Field = df_gis.MEAN_Field.apply(int)
    df_gis.columns = ['CITY_ENG','P_NUM','H_NUM','地価_数','地価_平均',
                      'X','Y','Near_FID','距離_駅','駅数','医療機関数']
    df_gis.loc[:,'駅数'] = df_gis.loc[:,'駅数'].apply(int)
    df_gis = df_gis.loc[:,['地価_平均','地価_数','駅数','医療機関数']]
    return df_gis

In [13]:
path_shapefile = "./japan_ver81"
df_gis = data_gis(path_shapefile)

### 3つのDataFrameを結合し、重複する特徴名をリネーム

In [14]:
df = pd.concat([df_stats,df_gis,df_census],axis=1,join='inner')
df.reset_index(inplace=True)
df.rename(columns={'index':'area_code'},inplace=True)

# Rename for identical column names (総数)
count = 0
list_newnames = []
for column in list(df.columns):
    if column == '総数':
        list_newnames.append('総数'+'.'+str(count+1))        
        count += 1
    else:
        list_newnames.append(column)
df.columns = list_newnames

In [15]:
def preprocessing(df):
    df_new = pd.DataFrame(df.loc[:,'area_code'])
    list_convert = ['300万円未満','300〜500万円未満','500〜700万円未満',
                    '700〜1000万円未満','1000〜1500万円未満','1500万円以上']
    list_amount = [300,400,600,850,1250,1500]
    df_new.loc[:,'平均年収'] = 0
    for i,item in enumerate(list_convert):
        df.loc[:,item] = df.loc[:,item] / (df.loc[:,'総数.3'] - df.loc[:,'不詳'])        
        df_new.loc[:,'平均年収'] += df.loc[:,item] * list_amount[i]

    func = lambda name,df: df.loc[:,name]/(df.loc[:,'卒業者'] - df.loc[:,'卒業者　不詳'])
    df_new.loc[:,'中卒率'] = func('卒業者　小学校・中学校',df)
    df_new.loc[:,'高卒率'] = func('卒業者　高校・旧中',df)
    df_new.loc[:,'短大卒率'] = func('卒業者　短大・高専',df)
    df_new.loc[:,'大卒率'] = func('卒業者　大学・大学院',df)
    
    df_new.loc[:,'地価_平均'] = df.loc[:,'地価_平均']
    df_new.loc[:,'平均世帯人員'] = df.loc[:,'平均世帯人員']
    df_new.loc[:,'住宅平均面積'] = df.loc[:,'住宅平均面積']
    df_new.loc[:,'駅密度'] = df.loc[:,'駅数'] / df.loc[:,'可住地面積']
    df_new.loc[:,'子供の有無'] = 1 / (1 + df.loc[:,'子はいない']/df.loc[:,'子がいる'])
    
    denom = df.loc[:,['Ａ 親族のみの世帯','Ｂ 非親族を含む世帯','Ｃ 単独世帯']].sum(axis=1)
    df_new.loc[:,'核家族率'] = df.loc[:,'1 核家族世帯'] / denom
    df_new.loc[:,'非核家族率'] = df.loc[:,'2 核家族以外の世帯'] / denom
    df_new.loc[:,'単身率'] = df.loc[:,'Ｃ 単独世帯'] / denom
    df_new.loc[:,'ルームシェア率'] = df.loc[:,'Ｂ 非親族を含む世帯'] / denom
    
    df_new.loc[:,'持ち家率'] = df.loc[:,'持ち家'] / df.loc[:,['持ち家','借家']].sum(axis=1)
    df_new.loc[:,'社宅率'] = df.loc[:,'給与住宅'] / df.loc[:,['借家']].sum(axis=1)
    df_new.loc[:,'UR率'] = df.loc[:,'都市再生機構(UR)・公社の借家'] / df.loc[:,['借家']].sum(axis=1)
    
    list_field = ['一般病院数（可住地面積100km2当たり）','完全失業率','刑法犯認知件数（人口千人当たり）',
                  '交通事故発生件数（人口10万人当たり）','百貨店，総合スーパー数（人口10万人当たり）',
                  '飲食店数（人口千人当たり）','第１次産業就業者比率（対就業者）',
                  '第２次産業就業者比率（対就業者）','第３次産業就業者比率（対就業者）']
    list_new = ['医療機関密度','失業率','犯罪率','交通事故発生率','商業施設密度',
                '飲食店密度','第１次産業比率','第２次産業比率','第３次産業比率']
    for i,item in enumerate(list_field):
        df_new.loc[:,list_new[i]] = df.loc[:,item]

    df_new.loc[:,'人口'] = df.loc[:,'住民基本台帳人口（総数）']
    df_new.loc[:,'外国人率'] = df.loc[:,'住民基本台帳人口（外国人）'] / df.loc[:,'住民基本台帳人口（総数）']
    df_new.loc[:,'出生死亡率'] = df.loc[:,'出生数'] / df.loc[:,'死亡数']
    df_new.loc[:,'通勤流入出率'] = df.loc[:,'他市区町村からの通勤者数'] / df.loc[:,'他市区町村への通勤者数']
    df_new.loc[:,'転入出率'] = df.loc[:,'転入者数'] / df.loc[:,'転出者数']
    df_new.loc[:,'結婚離婚率'] = df.loc[:,'婚姻件数'] / df.loc[:,'離婚件数']
    df_new.loc[:,'人口密度'] = df.loc[:,'住民基本台帳人口（総数）'] / df.loc[:,'可住地面積']
    df_new.loc[:,'男女比'] = df.loc[:,'住民基本台帳人口（日本人）（男）'] / df.loc[:,'住民基本台帳人口（日本人）（女）']
    df_new.loc[:,'労働力男女比'] = df.loc[:,'労働力人口（男）'] / df.loc[:,'労働力人口（女）']
    df_new.loc[:,'失業率男女比'] = df.loc[:,'完全失業者数（男）'] / df.loc[:,'完全失業者数（女）']
    df_new.loc[:,'役員率'] = df.loc[:,'役員数'] / df.loc[:,'労働力人口']
    df_new.loc[:,'労働力人口率'] = df.loc[:,'労働力人口'] / df.loc[:,'住民基本台帳人口（総数）']
    df_new.loc[:,'正規雇用率'] = df.loc[:,'雇用者数（正規の職員・従業員）'] / df.loc[:,'雇用者数（国勢調査結果）']
    df_new.loc[:,'フリーター率'] = df.loc[:,'雇用者数（パート・アルバイト・その他）'] / df.loc[:,'雇用者数（国勢調査結果）']
    df_new.loc[:,'幼稚園児童比率'] = df.loc[:,'幼稚園在園者数'] / df.loc[:,'幼稚園数']
    df_new.loc[:,'小学校児童比率'] = df.loc[:,'小学校児童数'] / df.loc[:,'小学校数']
    df_new.loc[:,'中学校生徒比率'] = df.loc[:,'中学校生徒数'] / df.loc[:,'中学校数']
    df_new.loc[:,'高校校生徒比率'] = df.loc[:,'高等学校生徒数'] / df.loc[:,'高等学校数']
    df_new.loc[:,'小学校教員比率'] = df.loc[:,'小学校教員数'] / df.loc[:,'小学校数']
    df_new.loc[:,'中学校教員比率'] = df.loc[:,'中学校教員数'] / df.loc[:,'中学校数']
    return df_new

In [16]:
df = preprocessing(df)
df.loc[:,'area_code'] = df.loc[:,'area_code'].apply(str).str.rjust(5,'0')
df.dropna(inplace=True)
df.rename(columns={'area_code':'JCODE'},inplace=True)
df = df.loc[df.loc[:,'商業施設密度'] > 0,:]

list_field = ['JCODE','平均年収','大卒率','平均世帯人員','駅密度','子供の有無',
              '持ち家率','医療機関密度','犯罪率','交通事故発生率','商業施設密度',
              '飲食店密度','幼稚園児童比率','転入出率','通勤流入出率','出生死亡率']
df_analysis = df.loc[:,list_field]

In [17]:
df_analysis.head()

,JCODE,平均年収,大卒率,平均世帯人員,駅密度,子供の有無,持ち家率,医療機関密度,犯罪率,交通事故発生率,商業施設密度,飲食店密度,幼稚園児童比率,転入出率,通勤流入出率,出生死亡率
0,01101,511.129221,0.310908,1.74,0.249534,0.565402,0.403646,126.6,30.17,1081.4,3.79,17.27,121.857143,1.096392,6.537111,0.885115
1,01102,463.139039,0.194001,2.06,0.058870,0.710734,0.523990,30.1,16.91,592.2,0.70,3.21,189.333333,1.085821,0.727477,0.738976
2,01103,464.676160,0.164519,2.03,0.053546,0.673677,0.418408,38.6,16.20,774.0,1.91,2.21,211.363636,1.070982,0.759058,0.885620
3,01104,442.931404,0.165038,1.94,0.068294,0.639020,0.366859,52.3,20.53,692.0,0.48,2.64,247.714286,1.042242,0.996348,0.837512
4,01105,453.249732,0.223094,1.95,0.105564,0.638527,0.418668,53.4,15.43,609.8,0.91,2.39,224.533333,1.122548,0.480085,0.860741


In [18]:
df_analysis.to_csv('./df_preprocessed.csv',index=False)